In [152]:
from __future__ import print_function
import pandas as pd 
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from ipywidgets import Dropdown
import numpy

plt.rcParams['figure.figsize'] = [10, 6]

In [153]:
# Source = https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv 
data = pd.read_csv("covid19.csv") 

# Generate a series of dates beginning from Jan 22, 2020 until the current end of the dataset
days_covered = len(data.columns) - 4
xseries = pd.date_range('2020-01-22', periods=days_covered, freq='D')

In [154]:
def get_data_subset(country, province):
    country_data = data[data['Country/Region'] == country]
    if province == 'All':
        return country_data
    else:
        province_data = country_data[country_data['Province/State'] == province]
        return province_data

In [155]:
def plot_linear(series, country, province):
    plt.plot(xseries, series)
    plt.ylabel('Infections')
    plt.title('Infections Over Time: ' + country + ' - '+ province)
    plt.xticks(rotation=45)
    plt.show()

In [156]:
def plot_log10(series, country, province):
    series_no_zero = [1 if y == 0 else y for y in series] # We can't take log of 0
    plt.plot(xseries, numpy.log10(series_no_zero))
    plt.ylabel('Log(Infections)')
    plt.title('Log(Infections) Over Time: ' + country + ' - '+ province)
    plt.xticks(rotation=45)
    plt.show()

In [157]:
def plot_daily_growth_rate(series, country, province):
    rates = [(series[i] - series[i-1])/series[i-1] * 100 if series[i-1] > 0 else None for i in range(1,len(series))]
    plt.plot(xseries[1:len(xseries)], rates)
    plt.ylabel('% growth in cases each day')
    plt.title('Daily % Growth Rate Over Time: ' + country + ' - '+ province)
    plt.xticks(rotation=45)
    plt.show()

In [158]:
def show_country_data(country, province):
    data = get_data_subset(country, province)

    sums = [None] * (len(data.columns) - 4)
    
    for col_idx in range(4, len(data.columns)):
        sums[col_idx-4] = data.iloc[:,col_idx].sum()
   
    plot_linear(sums, country, province)
    plot_log10(sums, country, province)
    plot_daily_growth_rate(sums, country, province)

In [159]:
# See https://stackoverflow.com/questions/45754356/jupyter-notebook-widgets-create-dependent-dropdowns

country_widget = Dropdown(options = sorted(data['Country/Region'].unique().tolist()))
province_widget = Dropdown(options = ['All'])

# Define a function that updates the content of province_widget based on what we select for country_widget
def update(change):
    country_subset = data[data['Country/Region'] == change.new]
    province_widget.options = ['All'] + sorted(country_subset['Province/State'].unique().tolist())
country_widget.observe(update, 'value')

def f(x,y):
    return

interact(show_country_data, 
         country=country_widget, 
         province=province_widget);

interactive(children=(Dropdown(description='country', options=('Afghanistan', 'Albania', 'Algeria', 'Andorra',…